In [355]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from fredapi import Fred
import sqlite3
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.dates as mdates
from pandas.tseries.offsets import Day
from decimal import Decimal
import requests
import seaborn as sns
color_pal = sns.color_palette()
from pandas.tseries.offsets import BDay
from multiprocessing import Pool
from tqdm import tqdm
from IPython.display import display
from IPython.display import HTML, display

In [356]:
# Function to get the next trading day
def get_next_trading_day():
    today = datetime.today()
    next_trading_day = today + BDay(1)
    return next_trading_day

In [357]:
# Parameters
ticker = "^GSPC"
start_date = "1950-01-01"
end_date = get_next_trading_day().strftime('%Y-%m-%d')

In [358]:
end_date

'2024-11-18'

In [359]:
data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [360]:
data

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
1950-01-03 00:00:00+00:00,16.660000,16.660000,16.660000,16.660000,16.660000,1260000
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000
...,...,...,...,...,...,...
2024-11-11 00:00:00+00:00,6001.350098,6001.350098,6017.310059,5986.689941,6008.859863,4333000000
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000


In [361]:
data

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
1950-01-03 00:00:00+00:00,16.660000,16.660000,16.660000,16.660000,16.660000,1260000
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000
...,...,...,...,...,...,...
2024-11-11 00:00:00+00:00,6001.350098,6001.350098,6017.310059,5986.689941,6008.859863,4333000000
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000


In [362]:
# # Add 'Adj Close' as a new column without replacing other columns
# data['Adj_Close'] = data['Adj Close']

In [363]:
# Calculate daily returns
data['Index_Returns'] = data['Adj Close'].pct_change()
data.dropna(inplace=True)

In [364]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,
Date,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927
...,...,...,...,...,...,...,...
2024-11-11 00:00:00+00:00,6001.350098,6001.350098,6017.310059,5986.689941,6008.859863,4333000000,0.000969
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893


In [365]:
# Check for NaNs in 'Index_Returns' and drop them
returns = data['Index_Returns'].dropna()

In [366]:
returns

Date
1950-01-04 00:00:00+00:00    0.011405
1950-01-05 00:00:00+00:00    0.004748
1950-01-06 00:00:00+00:00    0.002953
1950-01-09 00:00:00+00:00    0.005889
1950-01-10 00:00:00+00:00   -0.002927
                               ...   
2024-11-11 00:00:00+00:00    0.000969
2024-11-12 00:00:00+00:00   -0.002893
2024-11-13 00:00:00+00:00    0.000232
2024-11-14 00:00:00+00:00   -0.006050
2024-11-15 00:00:00+00:00   -0.013203
Name: Index_Returns, Length: 18840, dtype: float64

In [367]:
# Fit Markov Switching Model
model = MarkovRegression(returns, k_regimes=2, trend='c', switching_variance=True)
result = model.fit()
print(result.summary())

/opt/anaconda3/envs/msm_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                        Markov Switching Model Results                        
Dep. Variable:          Index_Returns   No. Observations:                18840
Model:               MarkovRegression   Log Likelihood               63442.623
Date:                Sat, 16 Nov 2024   AIC                        -126873.245
Time:                        14:21:37   BIC                        -126826.183
Sample:                             0   HQIC                       -126857.803
                              - 18840                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   5.65e-05     11.504      0.000       0.001       0.001
sigma2      3.999e-05   8.07e-07     49.571      0.0

In [368]:
# Add regime to the data
data.loc[returns.index, 'Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)

In [369]:
# # Create a scrollable display
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     html_output = data.to_html()

# display_html = f"""
# <div style="max-height: 300px; overflow-y: auto; border: 1px solid #ccc; padding: 10px;">
#     {html_output}
# </div>
# """
# display(HTML(display_html))

In [370]:
data['Vol_Regime']

Date
1950-01-04 00:00:00+00:00    0.0
1950-01-05 00:00:00+00:00    0.0
1950-01-06 00:00:00+00:00    0.0
1950-01-09 00:00:00+00:00    0.0
1950-01-10 00:00:00+00:00    0.0
                            ... 
2024-11-11 00:00:00+00:00    0.0
2024-11-12 00:00:00+00:00    0.0
2024-11-13 00:00:00+00:00    0.0
2024-11-14 00:00:00+00:00    0.0
2024-11-15 00:00:00+00:00    0.0
Name: Vol_Regime, Length: 18840, dtype: float64

In [371]:
# Extract smoothed probabilities and last known state probabilities
smoothed_probs = result.smoothed_marginal_probabilities

In [372]:
smoothed_probs

,0,1
Date,,
1950-01-04 00:00:00+00:00,0.964000,0.036000
1950-01-05 00:00:00+00:00,0.977845,0.022155
1950-01-06 00:00:00+00:00,0.983211,0.016789
1950-01-09 00:00:00+00:00,0.983449,0.016551
1950-01-10 00:00:00+00:00,0.980366,0.019634
...,...,...
2024-11-11 00:00:00+00:00,0.768027,0.231973
2024-11-12 00:00:00+00:00,0.790981,0.209019
2024-11-13 00:00:00+00:00,0.795531,0.204469


In [373]:
last_probs = smoothed_probs.iloc[-1].values

In [374]:
last_probs

array([0.7561809, 0.2438191])

In [375]:
# Extract transition probabilities from the model parameters
params = result.params
p_00 = params['p[0->0]']
p_10 = params['p[1->0]']
p_01 = 1 - p_00
p_11 = 1 - p_10

In [376]:
# Construct the transition matrix
transition_matrix = np.array([
    [p_00, p_01],
    [p_10, p_11]
])

In [377]:
transition_matrix

array([[0.98804773, 0.01195227],
       [0.03731172, 0.96268828]])

In [378]:
# Update state probabilities to predict the next day's regime
state_probs = np.dot(last_probs, transition_matrix)

In [379]:
state_probs

array([0.75624013, 0.24375987])

In [380]:
# Determine the most likely regime at t+1
regime_labels = smoothed_probs.columns.tolist()  # Should be [0, 1]
most_likely_regime = regime_labels[np.argmax(state_probs)]


In [381]:
most_likely_regime

0

In [382]:
# Assuming end_date is already defined as a string in the format 'YYYY-MM-DD'
end_date = datetime.strptime(end_date, '%Y-%m-%d')  # Convert to datetime object
predicted_date = (end_date + timedelta(days=0)).strftime('%Y-%m-%d')  # Increment and convert back to string

In [383]:
predicted_date

'2024-11-18'

In [384]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns,Vol_Regime
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,,
Date,,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405,0.0
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748,0.0
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953,0.0
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889,0.0
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927,0.0
...,...,...,...,...,...,...,...,...
2024-11-11 00:00:00+00:00,6001.350098,6001.350098,6017.310059,5986.689941,6008.859863,4333000000,0.000969,0.0
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893,0.0


In [385]:
def triangular_moving_average(series, n):
    smoothed_series = series.rolling(window=n//2, min_periods=1).mean()
    smoothed_series = smoothed_series.rolling(window=n//2, min_periods=1).mean()
    return smoothed_series

In [386]:
# Calculate 250-day triangular moving average
data['250_TMA'] = triangular_moving_average(data['Adj Close'], 250)

In [387]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Index_Returns,Vol_Regime,250_TMA
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,,,
Date,,,,,,,,,
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,16.850000,1890000,0.011405,0.0,16.850000
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,16.930000,2550000,0.004748,0.0,16.870000
1950-01-06 00:00:00+00:00,16.980000,16.980000,16.980000,16.980000,16.980000,2010000,0.002953,0.0,16.886667
1950-01-09 00:00:00+00:00,17.080000,17.080000,17.080000,17.080000,17.080000,2520000,0.005889,0.0,16.905000
1950-01-10 00:00:00+00:00,17.030001,17.030001,17.030001,17.030001,17.030001,2160000,-0.002927,0.0,16.918800
...,...,...,...,...,...,...,...,...,...
2024-11-11 00:00:00+00:00,6001.350098,6001.350098,6017.310059,5986.689941,6008.859863,4333000000,0.000969,0.0,5265.456761
2024-11-12 00:00:00+00:00,5983.990234,5983.990234,6009.919922,5960.080078,6003.600098,4243400000,-0.002893,0.0,5270.483643


In [388]:
required_columns = ['Vol_Regime', 'Adj Close', '250_TMA']
missing_columns = [col for col in required_columns if col not in data.columns]
if missing_columns:
    raise ValueError(f"Missing columns: {missing_columns}")

In [389]:
print(data.index)
print(data[['Vol_Regime', 'Adj Close', '250_TMA']].head())

DatetimeIndex(['1950-01-04 00:00:00+00:00', '1950-01-05 00:00:00+00:00',
               '1950-01-06 00:00:00+00:00', '1950-01-09 00:00:00+00:00',
               '1950-01-10 00:00:00+00:00', '1950-01-11 00:00:00+00:00',
               '1950-01-12 00:00:00+00:00', '1950-01-13 00:00:00+00:00',
               '1950-01-16 00:00:00+00:00', '1950-01-17 00:00:00+00:00',
               ...
               '2024-11-04 00:00:00+00:00', '2024-11-05 00:00:00+00:00',
               '2024-11-06 00:00:00+00:00', '2024-11-07 00:00:00+00:00',
               '2024-11-08 00:00:00+00:00', '2024-11-11 00:00:00+00:00',
               '2024-11-12 00:00:00+00:00', '2024-11-13 00:00:00+00:00',
               '2024-11-14 00:00:00+00:00', '2024-11-15 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=18840, freq=None)
Price                     Vol_Regime  Adj Close    250_TMA
Ticker                                    ^GSPC           
Date                                               

In [390]:
print(data[['Vol_Regime', 'Adj Close', '250_TMA']].isna().sum())

Price       Ticker
Vol_Regime            0
Adj Close   ^GSPC     0
250_TMA               0
dtype: int64


In [391]:
print(data.dtypes)

Price          Ticker
Adj Close      ^GSPC     float64
Close          ^GSPC     float64
High           ^GSPC     float64
Low            ^GSPC     float64
Open           ^GSPC     float64
Volume         ^GSPC       int64
Index_Returns            float64
Vol_Regime               float64
250_TMA                  float64
dtype: object


In [392]:
print(data.columns)
print(data.index)

MultiIndex([(    'Adj Close', '^GSPC'),
            (        'Close', '^GSPC'),
            (         'High', '^GSPC'),
            (          'Low', '^GSPC'),
            (         'Open', '^GSPC'),
            (       'Volume', '^GSPC'),
            ('Index_Returns',      ''),
            (   'Vol_Regime',      ''),
            (      '250_TMA',      '')],
           names=['Price', 'Ticker'])
DatetimeIndex(['1950-01-04 00:00:00+00:00', '1950-01-05 00:00:00+00:00',
               '1950-01-06 00:00:00+00:00', '1950-01-09 00:00:00+00:00',
               '1950-01-10 00:00:00+00:00', '1950-01-11 00:00:00+00:00',
               '1950-01-12 00:00:00+00:00', '1950-01-13 00:00:00+00:00',
               '1950-01-16 00:00:00+00:00', '1950-01-17 00:00:00+00:00',
               ...
               '2024-11-04 00:00:00+00:00', '2024-11-05 00:00:00+00:00',
               '2024-11-06 00:00:00+00:00', '2024-11-07 00:00:00+00:00',
               '2024-11-08 00:00:00+00:00', '2024-11-11 00:00:00+00:00'

In [393]:
# Define the four market regimes for 250 TMA
conditions = [
    (data['Vol_Regime'] == 1) & (data['Adj Close'] < data['250_TMA']),
    (data['Vol_Regime'] == 1) & (data['Adj Close'] >= data['250_TMA']),
    (data['Vol_Regime'] == 0) & (data['Adj Close'] < data['250_TMA']),
    (data['Vol_Regime'] == 0) & (data['Adj Close'] >= data['250_TMA']),
]
choices = [
    'Bearish High Variance',
    'Bullish High Variance',
    'Bearish Low Variance',
    'Bullish Low Variance'
]

ValueError: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.

In [279]:
# # Ensure 'predicted_date' is a valid date object or convert it
# if not isinstance(predicted_date, pd.Timestamp):
#     predicted_date = pd.Timestamp(predicted_date)

# # Check if 'predicted_date' already exists in the DataFrame
# if predicted_date not in data.index:
#     # Create a new row with NaN for all columns except 'Recursive_Predictions'
#     new_row = {'Recursive_Predictions': most_likely_regime, 'Recursive_Predicted_Regime_Label': None}

#     # Append the row using the Date index
#     data.loc[predicted_date] = pd.Series(new_row)

# # Access the last row using iloc[-1] and update the 'Recursive_Predictions' column
# data.iloc[-1, data.columns.get_loc('Vol_Regime')] = most_likely_regime